# Group Varaibles into Screens

Takes in a list of AL variables and spots out a suggested grouping.

Load libraries and functions.

In [1]:
import re
import spacy
import numpy as np
from numpy import unique
from numpy import where
from sklearn.cluster import AffinityPropagation

In [2]:
nlp = spacy.load('en_core_web_lg') # this takes a while to load

In [3]:
def reCase(text):
    # a quick and dirty way to pull words out of
    # snake_case, camelCase and the like.    
    output = re.sub("(\w|\d)(_|-)(\w|\d)","\\1 \\3",text.strip())
    output = re.sub("([a-z])([A-Z]|\d)","\\1 \\2",output)
    output = re.sub("(\d)([A-Z]|[a-z])","\\1 \\2",output)
    return output

In [ ]:
def cluster_screens(fields=[],damping=0.9):
    # Takes in a list (fields) and returns a suggested screen grouping
    # Set damping to value >= 0.5 < 1 to tune how related screens should be
    
    vec_mat = np.zeros([len(fields),300])
    for i in range(len(fields)):
        vec_mat[i] = [nlp(reCase(fields[i])).vector][0]

    # create model
    model = AffinityPropagation(damping=damping,random_state=4)
    # fit the model
    model.fit(vec_mat)
    # assign a cluster to each example
    yhat = model.predict(vec_mat)
    # retrieve unique clusters
    clusters = unique(yhat)

    screens = {}
    i=0
    for cluster in clusters:
        this_screen = where(yhat == cluster)[0]
        vars = []
        for screen in this_screen:
            vars.append(fields[screen])
        screens["screen_%s"%i]=vars
        i+=1

    return screens    

## Try it out

Use daming >= 0.5 and < 1 to tune the groupings.

In [13]:
fields= [
        "users1_name",
        "users1_birthdate",
        "users1_address_line_one",
        "users1_address_line_two",
        "users1_address_city",
        "users1_address_state",
        "users1_address_zip",
        "users1_phone_number",
        "users1_email",
        "plantiffs1_name",
        "defendants1_name",
        "petitioners1_name",
        "respondents1_name",
        "docket_number",
        "trial_court_county",
        "users1_signature",
        "signature_date"
        ]

cluster_screens(fields,damping=0.7)

{'screen_0': ['users1_name',
  'users1_birthdate',
  'users1_address_line_one',
  'users1_address_line_two',
  'users1_address_city',
  'users1_address_state',
  'users1_address_zip',
  'users1_phone_number',
  'users1_email',
  'users1_signature'],
 'screen_1': ['plantiffs1_name',
  'defendants1_name',
  'petitioners1_name',
  'respondents1_name'],
 'screen_2': ['docket_number'],
 'screen_3': ['trial_court_county'],
 'screen_4': ['signature_date']}